# Time and Dates

The `astropy.time` package provides functionality for manipulating times and dates. Specific emphasis is placed on supporting time scales (e.g. UTC, TAI, UT1, TDB) and time representations (e.g. JD, MJD, ISO 8601) that are used in astronomy and required to calculate, e.g., sidereal times and barycentric corrections. It uses Cython to wrap the C language ERFA time and calendar routines, using a fast and memory efficient vectorization scheme.

All time manipulations and arithmetic operations are done internally using two 64-bit floats to represent time. Floating point algorithms are used so that the Time object maintains sub-nanosecond precision over times spanning the age of the universe.

The basic way to use `astropy.time` is to create a `Time` object by supplying one or more input time values as well as the time format and time scale of those values. The input time(s) can either be a single scalar like `"2010-01-01 00:00:00"` or a list or a numpy array of values as shown below. In general any output values have the same shape (scalar or array) as the input.

In [ ]:
import numpy as np
from astropy.time import Time

In [ ]:
times = ['1999-01-01T00:00:00.123456789', '2010-01-01T00:00:00']
t = Time(times, format='isot', scale='utc')
t

In [ ]:
t[1]

The `format` argument specifies how to interpret the input values, e.g. ISO or JD or Unix time. The `scale` argument specifies the time scale for the values, e.g. UTC or TT or UT1. The `scale` argument is optional and defaults to UTC except for Time from epoch formats. We could have written the above as:

In [ ]:
t = Time(times, format='isot')

When the format of the input can be unambiguously determined then the format argument is not required, so we can simplify even further:

In [ ]:
t = Time(times)
t

Now let’s get the representation of these times in the JD and MJD formats by requesting the corresponding Time attributes:

In [ ]:
t.jd

In [ ]:
t.mjd

The default representation can be changed by setting the `format` attribute:



In [ ]:
t.format = 'fits'
t

In [ ]:
t.format = 'isot'
t

We can also convert to a different time scale, for instance from UTC to TT. This uses the same attribute mechanism as above but now returns a new `Time` object:

In [ ]:
t2 = t.tt
t2

In [ ]:
t2.jd

Note that both the ISO (ISOT) and JD representations of t2 are different than for t because they are expressed relative to the TT time scale. Of course, from the numbers or strings one could not tell; one format in which this information is kept is the `fits` format:

In [ ]:
print(t2.fits)

## Sidereal Time
Apparent or mean sidereal time can be calculated using `sidereal_time()`. The method returns a `Longitude` with units of hourangle, which by default is for the longitude corresponding to the location with which the `Time` object is initialized. Like the scale transformations, ERFA C-library routines are used under the hood, which support calculations following different IAU resolutions. Sample usage:

In [ ]:
t = Time('2006-01-15 21:24:37.5', scale='utc', location=('120d', '45d'))
t.sidereal_time('mean')  

In [ ]:
t.sidereal_time('apparent') 

## Time Deltas

Simple time arithmetic is supported using the TimeDelta class. The following operations are available:

* Create a TimeDelta explicitly by instantiating a class object
* Create a TimeDelta by subtracting two Times
* Add a TimeDelta to a Time object to get a new Time
* Subtract a TimeDelta from a Time object to get a new Time
* Add two TimeDelta objects to get a new TimeDelta
* Negate a TimeDelta or take its absolute value
* Multiply or divide a TimeDelta by a constant or array
* Convert TimeDelta objects to and from time-like Quantities

The `TimeDelta` class is derived from the `Time` class and shares many of its properties. One difference is that the time scale has to be one for which one day is exactly 86400 seconds. Hence, the scale cannot be UTC.

In [ ]:
t1 = Time('2010-01-01 00:00:00')
t2 = Time('2010-02-01 00:00:00')
dt = t2 - t1  # Difference between two Times
dt

In [ ]:
dt.sec

In [ ]:
from astropy.time import TimeDelta
dt2 = TimeDelta(50.0, format='sec')
t3 = t2 + dt2  # Add a TimeDelta to a Time
t3

## Timezones
When a Time object is constructed from a timezone-aware `datetime`, no timezone information is saved in the `Time` object. However, `Time` objects can be converted to timezone-aware datetime objects:

In [ ]:
from datetime import datetime
from astropy.time import Time, TimezoneInfo
import astropy.units as u

In [ ]:
utc_plus_one_hour = TimezoneInfo(utc_offset=1*u.hour)
dt_aware = datetime(2000, 1, 1, 0, 0, 0, tzinfo=utc_plus_one_hour)
t = Time(dt_aware)  # Loses timezone info, converts to UTC
print(t)            # will return UTC

In [ ]:
print(t.to_datetime(timezone=utc_plus_one_hour)) # to timezone-aware datetime